In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-twitter-political-classification-dataset/final_combined_enriched_v4.csv


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,precision_score
from sklearn.linear_model import LogisticRegression

# Data loading

In [3]:
df=pd.read_csv('/kaggle/input/indian-twitter-political-classification-dataset/final_combined_enriched_v4.csv')

# Data cleaning

In [4]:
df.drop_duplicates(inplace=True)

# Text preprocessing

In [5]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
def text_processing(text):
    text=text.lower()
    special_chars = "!$%^&*()-_=+[]{}|;:'\",.<>?/\\`~"
    for ch in special_chars:
        text=text.replace(ch,"")
    text_list=text.split(" ")
    final_list=[]
    for i in text_list:
        if i not in stop_words:
            final_list.append(i)
    stemmed_words = [ps.stem(word) for word in final_list]
    tweet=" ".join(stemmed_words)
    


    return tweet

In [6]:
df['tweet']=df['tweet'].apply(text_processing)

In [7]:
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['tweet'])

# Model training

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,df['type'],test_size=0.2,random_state=1)

In [9]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
print('Accuracy',accuracy_score(y_test,y_pred))
print('Precision',precision_score(y_test,y_pred))

Accuracy 0.8608973718397122
Precision 0.9968530622125393


In [10]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
print('Accuracy',accuracy_score(y_test,y_pred))
print('Precision',precision_score(y_test,y_pred))

Accuracy 0.9878085340261816
Precision 0.9939349384304356


In [11]:
def tweet_sentiment(text):
    text=text_processing(text)
    x=vectorizer.transform([text])
    y_pred=knn.predict(x)
    y_lr=lr.predict(x)
    return 'Political tweet' if y_lr else 'Non-political tweet'


# Prediction

In [12]:
tweet_sentiment("Rahul gandhi is youth leader isint it a sarcasm")

'Political tweet'